In [1]:
import cx_Oracle
import os
import pandas as pd
os.getcwd()



'/root/pcn_test/test_pcn'

In [2]:
class PCNDB():
    def __init__(self, dbUser, dbPass, dbSID):
        conStr = '{}/{}@{}'.format(dbUser, dbPass, dbSID)
        self.pcnCon = cx_Oracle.connect(conStr)
        self.cursor = self.pcnCon.cursor()

    def _exec_sql(self, sql_str):
        self.cursor.execute(sql_str)
        colList = [i[0] for i in self.cursor.description]
        rowsList = self.cursor.fetchall()
        return colList, rowsList

    def _trans_dict(self, colList, rowsList):
        if len(rowsList) == 0:
            return dict(zip(colList, [[None] for i in colList]))
        assert len(colList) == len(rowsList[0])
        resDict = dict(zip(colList, [list() for i in range(len(colList))]))
        for idx, valData in enumerate(rowsList):
            for col, val in zip(colList, valData):
                resDict[col].append(val)
        return resDict

    def _del_tCols(self, tCols=[]):
        tCols_str = ''
        if len(tCols) > 0:
            for col in tCols:
                addStr = '{}, '.format(col) if col != tCols[-1] else col
                tCols_str += addStr
        else:
            tCols_str = '*'
        return tCols_str

    def get_table_all_inf(self, tName, tCols=[]):
        tCols_str = self._del_tCols(tCols)
        selectSQL = 'SELECT {} FROM {}'.format(tCols_str, tName) 
        colList, rowsList = self._exec_sql(selectSQL)
        resDict = self._trans_dict(colList, rowsList)
        return resDict

    def _def_coninf(self, conDict):
        assert len(conDict.keys()) != 0
        conStrList = []
        for col, conData in conDict.items():
            assert len(conData) == 3
            conVal, conSymbol, conDtype = conData[0], conData[1], conData[2]
            conStr = ''
            if not isinstance(conVal, list):
                conVal = list(conVal)

            if conSymbol == '=':
                assert len(conVal) == 1
                valData = conVal[0]
                valStr = '\'{}\''.format(valData) if conDtype == 'str' else valData
                conStr = '{}={}'.format(col, valStr)
            elif conSymbol == 'in':
                assert len(conVal) > 0
                valStr = ''
                for d in conVal:
                    dataStr = '\'{}\''.format(d) if conDtype == 'str' else d
                    valStr += dataStr
                    if d != conVal[-1]:
                        valStr += ', '
                conStr = '{} in ({})'.format(col, valStr)
            elif conSymbol == '[]':
                assert len(conVal) == 2
                valLStr = '\'{}\''.format(conVal[0]) if conDtype == 'str' else conVal[0]
                valRStr = '\'{}\''.format(conVal[1]) if conDtype == 'str' else conVal[1]
                conStr = '({0} >= {1} and {0} <= {2})'.format(col, valLStr, valRStr)
            elif conSymbol == '()':
                assert len(conVal) == 2
                valLStr = '\'{}\''.format(conVal[0]) if conDtype == 'str' else conVal[0]
                valRStr = '\'{}\''.format(conVal[1]) if conDtype == 'str' else conVal[1]
                conStr = '({0} > {1} and {0} < {2})'.format(col, valLStr, valRStr)

            conStrList.append(conStr)
        return conStrList

    def get_con_str_inf(self, tName, conStr, tCols=[]):
        assert isinstance(conStr, str) and conStr != ''
        tCols_str = self._del_tCols(tCols)
        selectSQL = 'SELECT {} FROM {} WHERE {}'.format(tCols_str, tName, conStr) 
        colList, rowsList = self._exec_sql(selectSQL)
        resDict = self._trans_dict(colList, rowsList)
        return resDict

    ##conDict={'$Column_Name':[[], $symbol(=, in, [], ()), $dtype(str, int, float)]}
    ##conLogicList=['and' | 'or']
    def get_con_dict_inf(self, tName, conDict, conLogicList=[], tCols=[]):
        assert len(conDict.keys()) == len(conLogicList) + 1
        tCols_str = self._del_tCols(tCols)
        tCon_str = ''
        idx_conLogic = 0
        conStrList = self._def_coninf(conDict)

        for idx_con, conStr in enumerate(conStrList):
            if idx_conLogic == len(conLogicList):
                tCon_str += conStr
            elif idx_conLogic < len(conLogicList):
                tCon_str += '{} {} '.format(conStr, conLogicList[idx_conLogic])
            idx_conLogic += 1

        resDict = self.get_con_str_inf(tName, tCon_str, tCols)
        return resDict

    def get_station_inf(self, stnID, tCols=['OBJ_ID', 'NAME', 'FULL_NAME', 
                                            'STATION_TYPE', 'VOLTAGE_CLASS',
                                            'DSP_LEVEL', 'SITEDSP']):
        tName = 'T_SPC_SITE'
        conDict = {
            'OBJ_ID':[[stnID], '=', 'str']
        }
        res = self.get_con_dict_inf(tName=tName, conDict=conDict, tCols=tCols)
        return res
        
    def get_all_businesses(self, tCols=['BUSINESS_ID', 'CHANNEL_ID']):
        tName = 'T_BUSINESS_CHANNEL'
        tCols_str = self._del_tCols(tCols)
        selectSQL = 'SELECT {} FROM {}'.format(tCols_str, tName)
        cols, rows = self._exec_sql(selectSQL)
        res = self._trans_dict(cols, rows)
        return res

    def get_business_inf(self, b_id, tCols=['OBJ_ID', 'FULL_NAME', 'BUZ_TYPE', 
                                            'A_SITE_ID', 'Z_SITE_ID', 
                                            'DISPATCH_LEVEL', 'BUZ_RATE']):
        tName = 'T_BUZ'
        conDict = {
            'OBJ_ID':[[b_id], '=', 'str']
        }
        res = self.get_con_dict_inf(tName=tName, conDict=conDict, tCols=tCols)
        return res

    def get_channel_inf(self, c_id, tCols=['OBJ_ID', 'NAME', 'CHANNEL_TYPE', 
                                            'RATE', 'A_RES_ID', 'Z_RES_ID',
                                            'A_STATION', 'Z_STATION']):
        tName = 'T_CHANNEL_BASE'
        conDict = {
            'OBJ_ID':[[c_id], '=', 'str']
        }
        res = self.get_con_dict_inf(tName=tName, conDict=conDict, tCols=tCols)
        return res

    def get_all_channels(self, tCols=['OBJ_ID', 'NAME', 'CHANNEL_TYPE', 
                                            'RATE', 'A_RES_ID', 'Z_RES_ID',
                                            'A_STATION', 'Z_STATION']):
        tName = 'T_CHANNEL_BASE'
        tCols_str = self._del_tCols(tCols)
        selectSQL = 'SELECT {} FROM {}'.format(tCols_str, tName)
        cols, rows = self._exec_sql(selectSQL)
        res = self._trans_dict(cols, rows)
        return res

# 0.测试


In [3]:
pcnTestDB = PCNDB('PCN_TEST','784427618', '172.17.0.2:1521/LHR11G')

In [43]:
conDict={
    'A_STATION':[['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391', 'CAA7B1FA-7B47-4DD3-B69E-2361D66474AA-00773'], 'in', 'str'],
    'Z_STATION':[['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391', 'CAA7B1FA-7B47-4DD3-B69E-2361D66474AA-00773'], 'in', 'str']
}
conLogicList = ['and']
tCols = ['OBJ_ID', 'A_STATION', 'Z_STATIOn']
res = pcnTestDB.get_con_dict_inf(tName='T_CHANNEL_BASE', conDict=conDict, conLogicList=['and'], tCols=tCols)
res

{'OBJ_ID': ['2A9B3964-2821-4CE2-A6A0-F635259818DE-34357',
  '67269E55-4D3A-439D-84E8-CD7037B722CE-24684',
  '40796E70-B6E8-423A-9BA0-D7886F33C5DD-02498',
  '2A9B3964-2821-4CE2-A6A0-F635259818DE-34308',
  '57F4A1B4-FB14-4614-9C54-42A6699D434A-55097'],
 'A_STATION': ['CAA7B1FA-7B47-4DD3-B69E-2361D66474AA-00773',
  'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391',
  'CAA7B1FA-7B47-4DD3-B69E-2361D66474AA-00773',
  'CAA7B1FA-7B47-4DD3-B69E-2361D66474AA-00773',
  'CAA7B1FA-7B47-4DD3-B69E-2361D66474AA-00773'],
 'Z_STATION': ['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391',
  'CAA7B1FA-7B47-4DD3-B69E-2361D66474AA-00773',
  'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391',
  'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391',
  'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391']}

In [10]:
pcnTestDB.get_business_inf(b_id='B4409567-89B8-4B77-B1CC-F2BEC964941C-00357')

{'OBJ_ID': ['B4409567-89B8-4B77-B1CC-F2BEC964941C-00357'],
 'FULL_NAME': ['豫/220kVⅡ湛乔线距离保护(CSC-101DS)'],
 'BUZ_TYPE': ['1'],
 'A_SITE_ID': ['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00111'],
 'Z_SITE_ID': ['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00223'],
 'DISPATCH_LEVEL': ['3'],
 'BUZ_RATE': ['2M']}

In [8]:
pcnTestDB.get_station_inf(stnID='F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391')


{'OBJ_ID': ['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391'],
 'NAME': ['220kV徐庄变'],
 'FULL_NAME': ['豫.郑州/220kV徐庄变'],
 'STATION_TYPE': ['2'],
 'VOLTAGE_CLASS': ['8'],
 'DSP_LEVEL': ['3'],
 'SITEDSP': ['2']}

In [11]:
pd_bc = pd.read_csv('T_BUSINESS_CHANNEL.csv')

In [24]:
for idx, val in pd_bc['BUSINESS_ID'].items():
    if not isinstance(val, str):
        print(idx,val)
        print(type(val))


3387 nan
<class 'float'>
7027 nan
<class 'float'>
7596 nan
<class 'float'>


In [22]:
pd_bc.iloc[3387]

OBJ_ID               CDE3F008-D2DB-406A-9AA3-3EAE8060EC04-02654
OBJ_DISPIDX                                            60990302
BUSINESS_ID                                                 NaN
CHANNEL_ID           CD996599-B045-43B0-9785-F2B51C807217-29891
RESERVE_TYPE                                                  0
PROTECT_INTERFACE                                           NaN
TMS_SN                                                      NaN
SOURCE_OBJ_ID                                               NaN
TMS_UPDATE_TIME                                             NaN
BIND_TYPE                                                   NaN
Name: 3387, dtype: object

In [10]:
test_pd = pd.DataFrame(data=res)

In [11]:
test_pd

,OBJ_ID,NAME,CHANNEL_TYPE,RATE,A_RES_ID,Z_RES_ID,A_STATION,Z_STATION
0,40796E70-B6E8-423A-9BA0-D7886F33C5DD-15998,E_2M[110kV宁陵变:1框-11槽-CORE TRIB 1-11:1-4端口~时隙:1...,1,5,E173CFF7-1CC2-4289-B93F-5183F17D9C37-07183,E173CFF7-1CC2-4289-B93F-5183F17D9C37-15007,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00137,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00143
1,2A9B3964-2821-4CE2-A6A0-F635259818DE-34357,E_2M[孔河变XDM-100:1框-I7槽-I7-2M-60端口~时隙:1<>徐庄变XDM...,1,5,A5E7CDCD-9CAE-4473-A7FD-A7005ADDCA5A-55296,A5E7CDCD-9CAE-4473-A7FD-A7005ADDCA5A-45587,CAA7B1FA-7B47-4DD3-B69E-2361D66474AA-00773,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391
2,40796E70-B6E8-423A-9BA0-D7886F33C5DD-16014,E_2M[110kV宁陵变:1框-11槽-CORE TRIB 1-11:1-6端口~时隙:1...,1,5,E173CFF7-1CC2-4289-B93F-5183F17D9C37-06610,2C7DDB90-1FC9-403F-8CB7-F30A755A4F17-00047,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00137,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00129
3,B7F47344-7DC2-4969-8C08-FF10E7419E03-24839,E_2M[长乐变:1框-I7槽-I7-2M-2端口~时隙:1<>新密县供电公司XDM-100...,1,5,A5E7CDCD-9CAE-4473-A7FD-A7005ADDCA5A-96275,A5E7CDCD-9CAE-4473-A7FD-A7005ADDCA5A-90373,D656599F-45A0-48C5-A656-8C4F77ABD475-00069,6D764AE5-A5D6-48A6-BA42-5080BE590A3A-07069
4,6307DBF7-314A-4C27-AEB0-988E331A06DB-79156,(220kV天中变~110kV桓景变)马可尼622M光路01,2,3,8691246D-56FB-4B8B-AE65-ED64FADF5BF7-11039,5B2C3D02-2A4B-4F9D-B965-688B95E729E2-07050,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00211,531AD40B-761D-43F5-B6C3-3A248C254327-78129
...,...,...,...,...,...,...,...,...
33646,8553AB5E-C675-4AA3-ACEB-A0F3ABD41B51-62581,(220kV鲲鹏变~220kV辉县变)烽火622M光路02,2,3,BC58EEF1-6864-4FCE-B3F9-3365638FA846-15939,BC58EEF1-6864-4FCE-B3F9-3365638FA846-22565,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00156,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00151
33647,C005C4FF-2CE2-446A-91B9-6749AEE43020-34229,E_2M[<>],1,5,None,None,None,None
33648,C005C4FF-2CE2-446A-91B9-6749AEE43020-34459,E_2M[城西变2488 ADM+:1框-4槽-E1_63_TP端口~时隙:1<>新乡变24...,1,5,BC58EEF1-6864-4FCE-B3F9-3365638FA846-41599,CB5F5F84-EAA7-4199-8E26-9ECD37B15A71-03519,634E428A-9AC5-4DD6-BFBD-F8FEA5D219F9-01036,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00164
33649,68951367-750E-4B42-B1D3-28657AA997E5-75275,E_2M[110kV 凌庄变:1框-13槽-TRIB 1-22-6端口~时隙:1<>220k...,1,5,2C7DDB90-1FC9-403F-8CB7-F30A755A4F17-00604,2C7DDB90-1FC9-403F-8CB7-F30A755A4F17-00047,CAA7B1FA-7B47-4DD3-B69E-2361D66474AA-00441,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00129
